In [1]:
import gym
from gym.envs.registration import register

register(
    id='PortfolioEnv-v3',
    entry_point='ddqn.env:PortfolioEnv',
    kwargs={}
)

In [2]:
from ddqn.env import PortfolioEnv

env = gym.make('PortfolioEnv-v3')

env.observation_space

c:\coding\OOED\ooed\lib\site-packages\gym\utils\passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (50, 14)
  logger.warn(


Box(-inf, inf, (50, 14), float32)

In [3]:
from collections import defaultdict

import matplotlib.pyplot as plt
import torch
from tensordict.nn import TensorDictModule
from tensordict.nn.distributions import NormalParamExtractor
from torch import nn

from torchrl.collectors import SyncDataCollector
from torchrl.data.replay_buffers import ReplayBuffer
from torchrl.data.replay_buffers.samplers import SamplerWithoutReplacement
from torchrl.data.replay_buffers.storages import LazyTensorStorage
from torchrl.envs import (
    Compose,
    DoubleToFloat,
    ObservationNorm,
    StepCounter,
    TransformedEnv,
)
from torchrl.envs.libs.gym import GymEnv
from torchrl.envs.utils import check_env_specs, ExplorationType, set_exploration_type
from torchrl.modules import ProbabilisticActor, TanhNormal, ValueOperator
from torchrl.objectives import ClipPPOLoss
from torchrl.objectives.value import GAE
from tqdm import tqdm

In [4]:
device = "cpu" if not torch.has_cuda else "cuda:0"
device = "cpu"
num_cells = 256  
lr = 3e-4
max_grad_norm = 1.0
frame_skip = 1
frames_per_batch = 1000 // frame_skip
total_frames = 10_000 // frame_skip
sub_batch_size = 64  
num_epochs = 10  
clip_epsilon = 0.2 
gamma = 0.99
lmbda = 0.95
entropy_eps = 1e-4

C:\Users\Pasha\AppData\Local\Temp\ipykernel_5512\340062282.py:1: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  device = "cpu" if not torch.has_cuda else "cuda:0"


In [6]:
base_env = GymEnv("PortfolioEnv-v3", device=device, frame_skip=frame_skip)

In [12]:
columns = [x for x in base_env.src._data.columns.tolist() if x != 'reward'] 

In [18]:
### 

In [16]:
env = TransformedEnv(
    base_env,
    Compose(
        # normalize observations
        ObservationNorm(in_keys=columns),
        DoubleToFloat(
            in_keys=columns,
        ),
        StepCounter(),
    ),
)

In [29]:
env.transform[0].init_stats(num_iter=1000, reduce_dim=0, cat_dim=0, *columns)


TypeError: ObservationNorm.init_stats() got multiple values for argument 'reduce_dim'